In [1]:
import redis
import json

# Define connection variables
host = 'localhost'
port =  6379
password = 'mypassword'

# Connect to Redis
client = redis.Redis(host=host, port=port, password=password, decode_responses=True)
print('Connected to Redis')

client.flushdb()

Connected to Redis


# Personalized Product Recommendations
## Description
This use case demonstrates how to provide personalized product recommendations to users based on their past interactions and preferences.

## Redis Data Structures Used
- JSON
- Search

In [18]:
# Mock data for personalized product recommendations
mock_data = {
    'user:1001': {'preferences': ['electronics', 'books'], 'recent_views': ['product:2001', 'product:2002']},
    'user:1002': {'preferences': ['fashion', 'beauty'], 'recent_views': ['product:3001', 'product:3002']},
    'product:2001': {'category': 'electronics', 'name': 'Google Pixel', 'price': 499},
    'product:2002': {'category': 'books', 'name': 'Learn Python Programming', 'price': 25},
    'product:2003': {'category': 'electronics', 'name': 'iPhone', 'price': 799},
    'product:2004': {'category': 'books', 'name': 'Redis Deep Dive', 'price': 25},
    'product:3001': {'category': 'fashion', 'name': 'Zara Tshirt', 'price': 15},
    'product:3002': {'category': 'beauty', 'name': 'Dior Perfume', 'price': 185},
    'product:3003': {'category': 'fashion', 'name': 'Burberry Scarf', 'price': 115},
    'product:3004': {'category': 'beauty', 'name': 'Armani Perfume', 'price': 85}
}

# Ingest mock data into Redis
for key, value in mock_data.items():
    client.json().set(key, '$', json.dumps(value))

# Function to get personalized recommendations
def get_personalized_recommendations(user_id):
    user_data = json.loads(client.json().get(f'user:{user_id}', '$')[0])
    preferences = user_data['preferences']
    print(f"User ({user_id}) Category Preferences : {preferences}")
    recent_views = user_data['recent_views']
    print(f"User ({user_id}) Recent Views : {recent_views}")
    recommendations = []
    for pref in preferences:
        for key in client.scan_iter(match='product:*'):
            product = json.loads(client.json().get(key, '$')[0])
            if product['category'] == pref and key not in recent_views:
                recommendations.append(product)
    return recommendations

# Get recommendations for a user
user_id = 1002
recommendations = get_personalized_recommendations(user_id)
print(f'Recommendations for user {user_id}:')
for x in recommendations:print(x)

User (1002) Category Preferences : ['fashion', 'beauty']
User (1002) Recent Views : ['product:3001', 'product:3002']
Recommendations for user 1002:
{'category': 'fashion', 'name': 'Burberry Scarft', 'price': 115}
{'category': 'beauty', 'name': 'Armani Perfume', 'price': 85}


## How do you integrate Redisearch and Vector Search into this system ?